In [1]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import import_ipynb
from shared_functions import DataframeLoader, Shared
from libraries.DataframesUtils import CalculoDataframes
base_dir = 'C:\\Users\\Crisp\\Documents\\QualitySoftGroup\\Proyectos\\Liquidations-Incentives-Jupyter'
sub_direct = 'archivos-fuente-138'
folder_bbraun_source = os.path.join(base_dir, 'source', sub_direct)

importing Jupyter notebook from C:\Users\Crisp\Documents\QualitySoftGroup\Proyectos\Liquidations-Incentives-Jupyter\libraries\DataframesUtils.ipynb


In [2]:
dataframe_loader = DataframeLoader(
    base_dir=base_dir,
    sub_direct=sub_direct
)
shared = Shared()
print(dataframe_loader.cargar_dataframes)

<bound method DataframeLoader.cargar_dataframes of <shared_functions.DataframeLoader object at 0x0000022E4210A670>>


In [3]:
(
        df_resultado_detallado_previo,
        df_resultado_liquidaciones_previo,
        df_empleados,
        df_empleados_inactivos,
        df_tipos_empleados,
        df_clusters_empleado,
        df_zonas_empleado,
        df_salarios_variables,
        df_parrillas,
        df_venta_recaudo_real,
        df_venta_recaudo_presupuesto,
        df_venta_por_zona,
        df_recaudo_por_zona,
        df_rentabilidad,
        df_venta_otras_companias,
        df_recaudo_otras_companias,
        df_venta_recaudo_kams,
        df_rentabilidad_kam,
        df_venta_servicios_mtto,
        df_objetivos_cualitativos,
        df_incentivos_por_empleado,
        df_area_calculo_sba,
        df_factores_liquidacion,
        df_cluster_plan_real_recaudo,
        df_zona_plan_real_venta_ch,
        df_rentabilidad_zona,
        df_resultados_variables_cualitativas,
        df_zonas_clusters_empleado,
        df_centros_costos,
        df_empleados_centros_costos,
        df_centros_costos_grupos_productos_divisiones,
        df_parrillas_tipos_calculos,
        df_area_calculo_sba_centros_costos,
        df_renal_ambulatorio,
        codigos_ve_vc,
        _dataframes_entrada,
        fecha_liquidacion,
        meses_incentivos,
        OUTPUT_FOLDER
    ) = dataframe_loader.cargar_dataframes(base_dir, folder_bbraun_source)

In [4]:
fecha_liquidacion

'2024-05-01'

In [5]:
# maestra_resultados.to_excel('maestra_resultados_.xlsx')

In [6]:
maestra_resultados = pd.read_excel(
        io=os.path.join('maestra_resultados_.xlsx'),
        dtype={
            'Contexto': str,
            'Fecha': 'datetime64[ns]',
            'PorcentajeCumplimiento':float ,
            'Variable': str,
            'Real': float,
            'Presupuesto': float
        }
    )
maestra_resultados

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN
...,...,...,...,...,...,...,...
9138,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54
9139,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08
9140,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71
9141,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34


In [7]:
# maestra_resultados_centro_costo.to_excel('maestra_resultados_centro_costo.xlsx')

In [8]:
maestra_resultados_centro_costo = pd.read_excel(
        io=os.path.join('maestra_resultados_centro_costo_.xlsx'),
        dtype={
            'Fecha': 'datetime64[ns]',
            'GrupoProducto': str,
            'Division': str,
            'CodigoCentroCosto': str,
            'Real': float,
            'Presupuesto': float ,
            'Variable': str,
            'AreaCalculo': str,
            'Consecutivo': str,            
            'TipoEmpleado': str
        }
    )

maestra_resultados_centro_costo

,Unnamed: 0,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,AreaCalculo,Consecutivo,TipoEmpleado,PorcentajeCumplimiento
0,0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,NaN,NaN,NaN,0.970089
1,1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,NaN,NaN,NaN,0.903778
2,2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,NaN,NaN,NaN,1.118642
3,3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,NaN,NaN,NaN,0.000000
4,4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,NaN,NaN,NaN,1.173298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10890,15,5004293_3,2024-01-01,620,HC,323015090,7.666924e+08,1.011063e+09,VentaFacturadaGrupoClientesKAMSSub,NaN,NaN,NaN,0.758303
10891,16,5004293_3,2024-02-01,620,HC,323015090,2.062303e+09,2.148509e+09,VentaFacturadaGrupoClientesKAMSSub,NaN,NaN,NaN,0.959876
10892,17,5004293_3,2024-03-01,620,HC,323015090,3.146574e+09,3.412338e+09,VentaFacturadaGrupoClientesKAMSSub,NaN,NaN,NaN,0.922117
10893,18,5004293_3,2024-04-01,620,HC,323015090,4.339242e+09,4.454997e+09,VentaFacturadaGrupoClientesKAMSSub,NaN,NaN,NaN,0.974017


In [9]:
maestra_resultados_centro_costo['AreaCalculo'].fillna(0, inplace=True)
maestra_resultados_centro_costo['AreaCalculo'] = maestra_resultados_centro_costo['AreaCalculo'].astype(int)
maestra_resultados_centro_costo

,Unnamed: 0,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,AreaCalculo,Consecutivo,TipoEmpleado,PorcentajeCumplimiento
0,0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,0,NaN,NaN,0.970089
1,1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,0,NaN,NaN,0.903778
2,2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,0,NaN,NaN,1.118642
3,3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,0,NaN,NaN,0.000000
4,4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,0,NaN,NaN,1.173298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10890,15,5004293_3,2024-01-01,620,HC,323015090,7.666924e+08,1.011063e+09,VentaFacturadaGrupoClientesKAMSSub,0,NaN,NaN,0.758303
10891,16,5004293_3,2024-02-01,620,HC,323015090,2.062303e+09,2.148509e+09,VentaFacturadaGrupoClientesKAMSSub,0,NaN,NaN,0.959876
10892,17,5004293_3,2024-03-01,620,HC,323015090,3.146574e+09,3.412338e+09,VentaFacturadaGrupoClientesKAMSSub,0,NaN,NaN,0.922117
10893,18,5004293_3,2024-04-01,620,HC,323015090,4.339242e+09,4.454997e+09,VentaFacturadaGrupoClientesKAMSSub,0,NaN,NaN,0.974017


In [10]:
maestra_resultados_centro_costo[maestra_resultados_centro_costo['Variable'] == 'RentabilidadUnidadNegocioCM2COGSSUB']

,Unnamed: 0,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,AreaCalculo,Consecutivo,TipoEmpleado,PorcentajeCumplimiento
9738,0,1013583607_1,2024-03-01,805,AE,323007280,172273.580838,169713.102995,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.015087
9739,1,1013583607_1,2024-03-01,805,AE,323007290,172273.580838,169713.102995,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.015087
9740,2,1013583607_1,2024-04-01,805,AE,323007280,415179.066504,401140.061624,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.034998
9741,3,1013583607_1,2024-04-01,805,AE,323007290,415179.066504,401140.061624,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.034998
9742,4,1013583607_1,2024-05-01,805,AE,323007280,716717.704506,694280.875887,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.032317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9835,97,94460154_2,2024-05-01,A33,BA,323007337,23399.712828,20723.216486,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.129154
9836,98,94460154_2,2024-05-01,A34,BA,323007337,470369.985379,677503.485037,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,0.694269
9837,99,94460154_3,2024-03-01,A35,BA,323007337,64788.652349,110600.891998,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,0.585788
9838,100,94460154_3,2024-04-01,A35,BA,323007337,153573.270718,276453.474670,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,0.555512


## Euros

In [11]:
def convertir_a_euros(main):
    # Definir las tasas de cambio
    trm_marzo_2024 = 4253.92
    trm_abril_2024 = 4226.47
    trm_promedio_2024 = 4652.35

    # Crear una copia del DataFrame para evitar el SettingWithCopyWarning
    main = main.copy()

    # Convertir los valores de 'Real' y 'Presupuesto' a euros
    main['RealEuros'] = main['Real'] / trm_abril_2024
    main['PresupuestoEuros'] = main['Presupuesto'] / trm_promedio_2024
    main['PorcentajeCumplimientoEuros'] = (main['RealEuros']/main['PresupuestoEuros']).apply(lambda x: round(x, 2))

    # Renombrar las columnas originales 'Real' y 'Presupuesto'
    main = main.rename(columns={
        'Real': 'RealCOP', 'Presupuesto': 'PresupuestoCOP', 
        'PorcentajeCumplimiento': 'PorcentajeCumplimientoCOP',
        'PorcentajeCumplimientoEuros': 'PorcentajeCumplimiento'
    })

    # Renombrar las nuevas columnas para reflejar que están en euros
    main = main.rename(columns={'RealEuros': 'Real', 'PresupuestoEuros': 'Presupuesto'})
    return main[['Contexto', 'Variable', 'Fecha', 'Real', 'Presupuesto', 'PorcentajeCumplimiento', 'RealCOP', 'PresupuestoCOP', 'PorcentajeCumplimientoCOP']]

In [12]:
def unir_dataframes_por_columnas_comunes(df1, df2, how='inner'):
    """
    Une dos DataFrames basándose en sus columnas comunes con un tipo de unión especificado.
    
    Parámetros:
    - df1: Primer DataFrame.
    - df2: Segundo DataFrame.
    - how: Tipo de unión ('left', 'right', 'outer', 'inner'). Por defecto es 'inner'.
    
    Retorna:
    - DataFrame resultante de la unión.
    """
    # Encuentra las columnas comunes entre ambos DataFrames
    columnas_comunes = df1.columns.intersection(df2.columns)
    print(columnas_comunes)
    # Une los DataFrames basándose en las columnas comunes y el tipo de unión especificado
    df_unido = pd.merge(df1, df2, on=columnas_comunes.tolist(), how=how)
    
    return df_unido

In [13]:
# variables_convertir_euros = ['VentaFacturadaBraunPais', 'VentaFacturadaZonaPaisSub', 'VentaFacturadaDivisionSub', 'VentaFacturadaUnidadNegocioSub', 'VentaFacturadaGrupoClientesKAMSSub']
# variables_convertir_euros

In [14]:
# maestra_resultados_aplicar_euros = maestra_resultados[maestra_resultados['Variable'].isin(variables_convertir_euros)]
# maestra_resultados_aplicar_euros

In [15]:
# maestra_resultados[maestra_resultados['Variable'] == 'VentaFacturadaBraunPais']

In [16]:
# maestra_resultados_convertido_euros = maestra_resultados.copy()

# maestra_resultados_convertido_euros['RealCOP'] = 0
# maestra_resultados_convertido_euros['PresupuestoCOP'] = 0
# maestra_resultados_convertido_euros['PorcentajeCumplimientoCOP'] = 0

# maestra_resultados_convertido_euros.update(convertir_a_euros(maestra_resultados_aplicar_euros))
# maestra_resultados_convertido_euros

In [17]:
# maestra_resultados_convertido_euros[maestra_resultados_convertido_euros['Variable'] == 'VentaFacturadaBraunPais']

In [18]:
# # Renombrar las nuevas columnas para reflejar que están en euros
# main = main.rename(columns={'RealEuros': 'Real', 'PresupuestoEuros': 'Presupuesto'})

In [19]:
# # Renombrar las columnas originales 'Real' y 'Presupuesto'
# main = main.rename(columns={
#     'Real': 'RealCOP', 'Presupuesto': 'PresupuestoCOP', 
#     'PorcentajeCumplimiento': 'PorcentajeCumplimientoCOP',
#     'PorcentajeCumplimientoEuros': 'PorcentajeCumplimiento'
# })

In [20]:
# maestra_resultados_convertidos = maestra_resultados.drop(columns=['Unnamed: 0']).merge(
#     maestra_resultados_convertido_euros,
#     on = ['Contexto', 'Variable', 'Fecha'],
#     how='left'
# )
# maestra_resultados_convertidos

In [21]:
# maestra_resultados_convertidos[maestra_resultados_convertidos['Variable'] == 'VentaFacturadaBraunPais']

In [22]:
# maestra_resultados = maestra_resultados_convertido_euros.copy()
# maestra_resultados

## Se recalculan los porcentajes de cumplimiento para darle manejo a negativos

In [23]:
df_parrillas[df_parrillas['Variable'] == 'VentaCobradaBraunPais']

,CodigoEmpleado,Variable,Contexto,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
0,5004523,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
9,1144160710,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
17,1111128,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
39,5004589,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
45,5032450,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
...,...,...,...,...,...,...,...
1305,5041369,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
1319,5005917,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
1324,5041312,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
1332,1111118,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN


In [24]:
calculo_dataframe = CalculoDataframes()

variables_convertir_euros = ['VentaFacturadaBraunPais', 'VentaFacturadaZonaPaisSub', 'VentaFacturadaDivisionSub', 'VentaFacturadaUnidadNegocioSub', 'VentaFacturadaGrupoClientesKAMSSub']
# Convertir las variables a euros en 'maestra_resultados'
maestra_resultados = calculo_dataframe.convertir_valores_a_euros(maestra_resultados, variables_convertir_euros)

# Convertir las variables a euros en 'maestra_resultados_centro_costo'
maestra_resultados_centro_costo = calculo_dataframe.convertir_valores_a_euros(maestra_resultados_centro_costo, variables_convertir_euros)

# ## Se recalculan los porcentajes de cumplimiento para darle manejo a negativos

In [25]:
def recalcular_porcentajes_cumplimiento(row):
    response = row['PorcentajeCumplimiento']
    if not pd.isnull(row['Real']) and not pd.isnull(row['Presupuesto']):
        if row['Presupuesto'] == 0:
            response = 0
        elif row['Real'] - row['Presupuesto'] < 0:
            response = 1 + ((row['Real'] - row['Presupuesto']) / abs(row['Presupuesto']))
        elif row['Real'] - row['Presupuesto'] > 0:
            response = 1 + abs((row['Real'] - row['Presupuesto']) / abs(row['Presupuesto']))
    return round(response, 2)

maestra_resultados['PorcentajeCumplimiento'] = maestra_resultados.apply(recalcular_porcentajes_cumplimiento, axis=1)
maestra_resultados

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
9138,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0
9139,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0
9140,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0
9141,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0


### Se calculan los porcentajes de cumplimiento para Maestra de Centro de Costos

In [26]:
# maestra_resultados_centro_costo['PorcentajeCumplimiento'] = 0
maestra_resultados_centro_costo['PorcentajeCumplimiento'] = maestra_resultados_centro_costo.apply(recalcular_porcentajes_cumplimiento, axis=1)
# maestra_resultados_centro_costo.drop(columns={'AreaCalculo', 'Consecutivo', 'TipoEmpleado'})

## Se une la maestra de resultados con la parrilla

In [27]:
maestra_resultados[
     (maestra_resultados['Fecha']=='2024-05-01') &
    (maestra_resultados['Contexto']=='1111117')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP
514,514,1111117,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
724,24,1111117,2024-05-01,0.88,VentaFacturadaZonaCargoIndividual,3.179236e+08,3.617851e+08,0.0,0.0,0.0
2054,24,1111117,2024-05-01,0.79,VentasCobradaZonaCargoIndividual,3.044938e+08,3.835532e+08,0.0,0.0,0.0
4350,29,1111117,2024-05-01,0.86,VentaCobradaBraunCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0
5443,24,1111117,2024-05-01,0.88,VentaFacturadaPortafolioCluster,3.179236e+08,3.617851e+08,0.0,0.0,0.0
6797,39,1111117,2024-05-01,0.94,VentaFacturadaZonaPais,5.855320e+10,6.258874e+10,0.0,0.0,0.0
7117,4,1111117,2024-05-01,3.37,VentaFacturadaPortafolioRenal,1.814526e+09,5.383562e+08,0.0,0.0,0.0
7676,24,1111117,2024-05-01,0.86,VentaCobradaPortafolioCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0
9127,4,1111117,2024-05-01,1.04,VentaCobradaPortafolioRenal,8.571691e+08,8.276971e+08,0.0,0.0,0.0


In [28]:
# df_parrillas[df_parrillas['Variable'] == 'RentabilidadDivisionCM2COGsSub']

In [29]:
df_parrillas_tipos_calculos.head()

,TipoEmpleado,AreaCalculo,CodigoRubro,NombreRubro,TipoCalculo
0,AD,1,2,EBITCOGSPaisSUB,F
1,AD,1,3,ObjetivosCualitativos,F
2,AD,1,1,VentaFacturadaBraunPaisSUB,F
3,AD,2,2,EBITCOGSPaisSUB,F
4,AD,2,3,ObjetivosCualitativos,F


In [30]:
df_parrillas[df_parrillas['CodigoEmpleado'] == '5004523']

,CodigoEmpleado,Variable,Contexto,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
0,5004523,VentaCobradaBraunPais,Pais,0.30,0,Venta cobrada país,NaN
1,5004523,ObjetivosCualitativos,5004523,0.15,0,Objetivos cualitativos,NaN
2,5004523,VentaFacturadaPortafolioClusterSub,5004523_1,0.06,1,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Neuro)
3,5004523,VentaFacturadaPortafolioClusterSub,5004523_2,0.06,2,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Ortho)
4,5004523,VentaFacturadaPortafolioClusterSub,5004523_3,0.06,3,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Surgical Assets)
5,5004523,VentaFacturadaPortafolioClusterSub,5004523_4,0.06,4,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Operating Room Soluti...
6,5004523,VentaFacturadaPortafolioClusterSub,5004523_5,0.03,5,VentaFacturadaPortafolioClusterSub,Patient Care (Desinfeccion)
7,5004523,VentaFacturadaPortafolioClusterSub,5004523_6,0.03,6,VentaFacturadaPortafolioClusterSub,Patient Care (Heridas+Ostomia)
8,5004523,VentaCobradaBraunCluster,5004523,0.25,0,Venta Cobrada presupuesto asignado total B. Br...,NaN


In [31]:
maestra_resultados[maestra_resultados['Variable'] == 'VentaCobradaBraunPais']

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP
3400,0,Pais,2024-01-01,0.95,VentaCobradaBraunPais,1.670438e+10,1.749805e+10,0.0,0.0,0.0
3401,1,Pais,2024-02-01,0.88,VentaCobradaBraunPais,3.221882e+10,3.663783e+10,0.0,0.0,0.0
3402,2,Pais,2024-03-01,0.86,VentaCobradaBraunPais,4.803727e+10,5.604000e+10,0.0,0.0,0.0
3403,3,Pais,2024-04-01,0.87,VentaCobradaBraunPais,6.413254e+10,7.395302e+10,0.0,0.0,0.0
3404,4,Pais,2024-05-01,0.91,VentaCobradaBraunPais,8.319083e+10,9.187600e+10,0.0,0.0,0.0


In [32]:
maestra_resultados_centro_costo[maestra_resultados_centro_costo['Variable'] == 'VentaFacturadaPortafolioClusterSub']

,Unnamed: 0,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,AreaCalculo,Consecutivo,TipoEmpleado,PorcentajeCumplimiento,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP
4372,0,1111111_1,2024-01-01,805,AE,323007332,7.798772e+07,1.288839e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,6.05,0.0,0.0,0.0
4373,1,1111111_1,2024-01-01,805,AE,323007333,7.798772e+07,1.288839e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,6.05,0.0,0.0,0.0
4374,2,1111111_1,2024-01-01,805,AE,323010170,7.798772e+07,1.288839e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,6.05,0.0,0.0,0.0
4375,3,1111111_1,2024-01-01,M06,AE,323007332,2.362315e+06,1.075307e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.22,0.0,0.0,0.0
4376,4,1111111_1,2024-01-01,M06,AE,323007333,2.362315e+06,1.075307e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.22,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8573,4201,53069050_3,2024-05-01,610,HC,323015095,1.341899e+08,1.850790e+08,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.73,0.0,0.0,0.0
8574,4202,79625102_1,2024-02-01,A35,BA,323007330,2.341397e+08,2.680889e+08,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.87,0.0,0.0,0.0
8575,4203,79625102_1,2024-03-01,A35,BA,323007330,3.244427e+08,5.808592e+08,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.56,0.0,0.0,0.0
8576,4204,79625102_1,2024-04-01,A35,BA,323007330,4.592916e+08,9.383110e+08,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.49,0.0,0.0,0.0


In [33]:
df_parrillas[
     (df_parrillas['Variable'] == 'RentabilidadUnidadNegocioCM2COGSSUB') 
#     (df_parrillas['CodigoEmpleado'] == '5036358')
]

,CodigoEmpleado,Variable,Contexto,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
36,53069050,RentabilidadUnidadNegocioCM2COGSSUB,53069050_1,0.04,1,RentabilidadUnidadNegocioCM2COGSSUB,Grupo Local 640 Enteral Diets
37,53069050,RentabilidadUnidadNegocioCM2COGSSUB,53069050_2,0.10,2,RentabilidadUnidadNegocioCM2COGSSUB,Grupo Local 680 Parenteral Nutrition
38,53069050,RentabilidadUnidadNegocioCM2COGSSUB,53069050_3,0.06,3,RentabilidadUnidadNegocioCM2COGSSUB,Grupo Local 610 IV Drugs & LVP
254,5004297,RentabilidadUnidadNegocioCM2COGSSUB,5004297_1,0.10,1,RentabilidadUnidadNegocioCM2COGSSUB,ORS (003)
255,5004297,RentabilidadUnidadNegocioCM2COGSSUB,5004297_2,0.09,2,RentabilidadUnidadNegocioCM2COGSSUB,MIS (801)
256,5004297,RentabilidadUnidadNegocioCM2COGSSUB,5004297_3,0.01,3,RentabilidadUnidadNegocioCM2COGSSUB,iVT 006
302,91285756,RentabilidadUnidadNegocioCM2COGSSUB,91285756_1,0.06,1,RentabilidadUnidadNegocioCM2COGSSUB,8 IV Systems & Drainage
303,91285756,RentabilidadUnidadNegocioCM2COGSSUB,91285756_2,0.13,2,RentabilidadUnidadNegocioCM2COGSSUB,12 Automated Infusion Systems & Disposablesm+ ...
304,91285756,RentabilidadUnidadNegocioCM2COGSSUB,91285756_3,0.01,3,RentabilidadUnidadNegocioCM2COGSSUB,SBA02 Drugs
360,5004306,RentabilidadUnidadNegocioCM2COGSSUB,5004306_1,0.10,1,RentabilidadUnidadNegocioCM2COGSSUB,ORS (003)


In [34]:
maestra_resultados[maestra_resultados['Contexto'] == '5002711_1'].merge(
    df_parrillas[df_parrillas['Contexto'] == '5002711_1'],
    on=['Variable', 'Contexto'],
    how='left'
)

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
0,0,5002711_1,2024-01-01,0.99,VentaCobradaZonaPaisSub,3.784505e+09,3.817880e+09,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
1,1,5002711_1,2024-02-01,0.99,VentaCobradaZonaPaisSub,7.780378e+09,7.846545e+09,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
2,2,5002711_1,2024-03-01,0.94,VentaCobradaZonaPaisSub,1.117600e+10,1.187179e+10,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
3,3,5002711_1,2024-04-01,0.96,VentaCobradaZonaPaisSub,1.488131e+10,1.546509e+10,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
4,4,5002711_1,2024-05-01,0.99,VentaCobradaZonaPaisSub,1.878196e+10,1.905839e+10,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
5,0,5002711_1,2024-01-01,1.37,VentaFacturadaZonaPaisSub,7.803963e+05,5.712712e+05,3.298322e+09,2.657754e+09,1.24,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care
6,9,5002711_1,2024-02-01,1.09,VentaFacturadaZonaPaisSub,1.370481e+06,1.254635e+06,5.792297e+09,5.837001e+09,0.99,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care
7,18,5002711_1,2024-03-01,1.11,VentaFacturadaZonaPaisSub,2.336360e+06,2.103221e+06,9.874553e+09,9.784922e+09,1.01,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care
8,27,5002711_1,2024-04-01,1.18,VentaFacturadaZonaPaisSub,3.266975e+06,2.761944e+06,1.380777e+10,1.284953e+10,1.07,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care
9,36,5002711_1,2024-05-01,1.17,VentaFacturadaZonaPaisSub,4.218864e+06,3.612253e+06,1.783090e+10,1.680546e+10,1.06,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care


### Maestra de Resultados por Empleado

In [35]:
maestra_resultados_por_empleado = maestra_resultados.merge(
    df_parrillas,
    how='left',
    indicator=True
)
maestra_resultados_por_empleado

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10497,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,1010166177,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both
10498,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,1010166177,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both
10499,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,1010166177,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both
10500,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,1010166177,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both


In [36]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge
514,514,1111117,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,1111117,0.15,0.0,Objetivos cualitativos,NaN,both
724,24,1111117,2024-05-01,0.88,VentaFacturadaZonaCargoIndividual,3.179236e+08,3.617851e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
2054,24,1111117,2024-05-01,0.79,VentasCobradaZonaCargoIndividual,3.044938e+08,3.835532e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
5415,29,1111117,2024-05-01,0.86,VentaCobradaBraunCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
6508,24,1111117,2024-05-01,0.88,VentaFacturadaPortafolioCluster,3.179236e+08,3.617851e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
7972,39,1111117,2024-05-01,0.94,VentaFacturadaZonaPais,5.855320e+10,6.258874e+10,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
8292,4,1111117,2024-05-01,3.37,VentaFacturadaPortafolioRenal,1.814526e+09,5.383562e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
8851,24,1111117,2024-05-01,0.86,VentaCobradaPortafolioCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
10486,4,1111117,2024-05-01,1.04,VentaCobradaPortafolioRenal,8.571691e+08,8.276971e+08,0.0,0.0,0.0,1111117,0.25,0.0,VentaCobradaPortafolioRenal,NaN,both


In [37]:
df_empleados[df_empleados['CodigoEmpleado'] == '1111117']

,TipoEmpleado,CodigoEmpleado,Identificacion,CorreoElectronico,Nombre,Apellidos,SalarioVariable,FechaIngreso,FechaRetiro,AreaCalculo
102,VC,1111117,1014226605,julieth.pedraza@bbraun.com,Julieth,Pedraza,2500000.0,2023-01-01,NaT,5


In [38]:
maestra_resultados_por_empleado[maestra_resultados_por_empleado['Variable'] == 'VentaFacturadaGrupoClientesKAMSSub']

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge
8788,0,kam_2_1,2024-01-01,1.04,VentaFacturadaGrupoClientesKAMSSub,1.579665e+05,1.517270e+05,6.676407e+08,7.058871e+08,0.95,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8789,1,kam_2_1,2024-02-01,0.91,VentaFacturadaGrupoClientesKAMSSub,3.274076e+05,3.583340e+05,1.383778e+09,1.667095e+09,0.83,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8790,2,kam_2_1,2024-03-01,0.88,VentaFacturadaGrupoClientesKAMSSub,5.873638e+05,6.650162e+05,2.482475e+09,3.093888e+09,0.80,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8791,3,kam_2_1,2024-04-01,0.93,VentaFacturadaGrupoClientesKAMSSub,8.059016e+05,8.651667e+05,3.406119e+09,4.025058e+09,0.85,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8792,4,kam_2_1,2024-05-01,0.91,VentaFacturadaGrupoClientesKAMSSub,1.036601e+06,1.136338e+06,4.381161e+09,5.286644e+09,0.83,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8793,5,kam_2_2,2024-01-01,1.49,VentaFacturadaGrupoClientesKAMSSub,5.477026e+05,3.682217e+05,2.314848e+09,1.713096e+09,1.35,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both
8794,6,kam_2_2,2024-02-01,1.30,VentaFacturadaGrupoClientesKAMSSub,1.130694e+06,8.696299e+05,4.778845e+09,4.045823e+09,1.18,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both
8795,7,kam_2_2,2024-03-01,1.16,VentaFacturadaGrupoClientesKAMSSub,1.873040e+06,1.613908e+06,7.916346e+09,7.508463e+09,1.05,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both
8796,8,kam_2_2,2024-04-01,1.22,VentaFacturadaGrupoClientesKAMSSub,2.569475e+06,2.099647e+06,1.085981e+10,9.768292e+09,1.11,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both
8797,9,kam_2_2,2024-05-01,1.21,VentaFacturadaGrupoClientesKAMSSub,3.337287e+06,2.757745e+06,1.410494e+10,1.283000e+10,1.10,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both


In [39]:
maestra_resultados_por_empleado = maestra_resultados_por_empleado.merge(
    df_empleados[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso']],
    on='CodigoEmpleado',
    how='left'
)
maestra_resultados_por_empleado

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10497,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10498,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10499,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10500,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01


In [40]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Variable'] == 'ObjetivosCualitativos') &
    (maestra_resultados_por_empleado['Contexto'] == '5004293')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso
625,625,5004293,2024-01-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17
626,626,5004293,2024-02-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17
627,627,5004293,2024-03-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17
628,628,5004293,2024-04-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17
629,629,5004293,2024-05-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17


In [41]:
# Se añaden líneas para filtrar registros vacíos ('Fecha' y 'CodigoEmpleado')
maestra_resultados_por_empleado.dropna(subset=[
        'Fecha'
    ], inplace=True)
maestra_resultados_por_empleado['CodigoEmpleado'] = maestra_resultados_por_empleado['CodigoEmpleado'].astype(str)
maestra_resultados_por_empleado

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10497,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10498,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10499,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10500,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01


In [48]:
df_salarios_variables[df_salarios_variables['CodigoEmpleado'] == '16659046']

,CodigoEmpleado,Fecha,SalarioVariable,PromedioSalarioVariable
536,16659046,2024-01-01,1712000.0,1712000.0
537,16659046,2024-02-01,1712000.0,1712000.0
538,16659046,2024-03-01,1712000.0,1712000.0
539,16659046,2024-04-01,3149000.0,2071250.0
540,16659046,2024-05-01,3149000.0,2286800.0


In [49]:
maestra_resultados_por_empleado[maestra_resultados_por_empleado['CodigoEmpleado'] == '16659046']

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,SalarioVariable
550,550,16659046,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,1712000.0
551,551,16659046,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,1712000.0
552,552,16659046,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,1712000.0
553,553,16659046,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,2071250.0
554,554,16659046,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,2286800.0
10123,60,16659046_1,2024-03-01,1.05,VentaFacturadaBraunPaisSUB,1.120332e+07,1.062285e+07,0.0,0.0,0.0,...,1.0,VentaFacturadaBraunPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,1712000.0
10124,61,16659046_1,2024-04-01,1.09,VentaFacturadaBraunPaisSUB,1.547450e+07,1.420999e+07,0.0,0.0,0.0,...,1.0,VentaFacturadaBraunPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,2071250.0
10125,62,16659046_1,2024-05-01,1.06,VentaFacturadaBraunPaisSUB,1.964573e+07,1.858577e+07,0.0,0.0,0.0,...,1.0,VentaFacturadaBraunPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,2286800.0
10390,18,16659046_1,2024-03-01,1.18,EBITCOGSPaisSUB,2.980374e+06,2.534506e+06,0.0,0.0,0.0,...,1.0,EBITCOGSPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,1712000.0
10391,19,16659046_1,2024-04-01,1.12,EBITCOGSPaisSUB,3.789609e+06,3.390361e+06,0.0,0.0,0.0,...,1.0,EBITCOGSPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17,2071250.0


In [44]:
# TODO Hacer el merge con la tabla de promedio variable
maestra_resultados_por_empleado = maestra_resultados_por_empleado.merge(
    df_salarios_variables[['CodigoEmpleado', 'Fecha', 'PromedioSalarioVariable']].rename(
        columns={'PromedioSalarioVariable': 'SalarioVariable'}
    ),
    on=['CodigoEmpleado', 'Fecha'],
    how='left'
)
maestra_resultados_por_empleado

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,SalarioVariable
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10471,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,...,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01,2500000.0
10472,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,...,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01,2500000.0
10473,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,...,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01,2500000.0
10474,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,...,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01,2500000.0


In [ ]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117')
]

### Maestra de Resultados por Empleado y Centro de Costos

In [ ]:
maestra_resultados_centro_costo.head()

In [ ]:
df_parrillas

In [ ]:
maestra_resultados_por_empleado_centro_costo = maestra_resultados_centro_costo.drop(columns={'AreaCalculo', 'Consecutivo', 'TipoEmpleado', 'Unnamed: 0'})

In [ ]:
maestra_resultados_por_empleado_centro_costo = maestra_resultados_por_empleado_centro_costo.merge(
    df_parrillas,
    on=['Contexto', 'Variable'],
    how='left'
)
maestra_resultados_por_empleado_centro_costo.head()

In [ ]:
maestra_resultados_por_empleado_centro_costo[
    (maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '5002899')
]   

In [ ]:
maestra_resultados_por_empleado_centro_costo[
    (maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '1032450706') &
    (maestra_resultados_por_empleado_centro_costo['Fecha'] == '2023-01-01')
]

In [ ]:
# maestra_resultados_por_empleado_centro_costo = maestra_resultados_centro_costo.drop(columns=['Unnamed: 0' , 'Consecutivo']).merge(
#     df_parrillas_tipos_calculos,
#     on=['TipoEmpleado', 'AreaCalculo'],
#     how='left'
# )
# maestra_resultados_por_empleado_centro_costo.head()

In [ ]:
# maestra_resultados_por_empleado_centro_costo = maestra_resultados_por_empleado_centro_costo.merge(
#     df_empleados_centros_costos.rename(columns={'PorcentajeAsignacion': 'Porcentaje'})
# )

In [ ]:
# maestra_resultados_por_empleado_centro_costo[maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '1032450706']

In [ ]:
df_empleados[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso', 'FechaRetiro']][df_empleados['CodigoEmpleado'] == '5005463']

In [ ]:
df_empleados[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso', 'FechaRetiro']][df_empleados['CodigoEmpleado'] == '1098612757']

In [ ]:
df_empleados_inactivos[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso', 'FechaRetiro']][df_empleados_inactivos['CodigoEmpleado'] == '1032450706']

In [ ]:
df_empleados_inactivos[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso', 'FechaRetiro']][df_empleados_inactivos['CodigoEmpleado'] == '5005463']

In [ ]:
df_empleados_totales = pd.concat([df_empleados, df_empleados_inactivos])

In [ ]:
maestra_resultados_por_empleado_centro_costo = maestra_resultados_por_empleado_centro_costo.merge(
    df_empleados_totales[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso']],
    on=['CodigoEmpleado'],
    how='left'
)
maestra_resultados_por_empleado_centro_costo.head()

In [ ]:
# Se añaden líneas para filtrar registros vacíos ('Fecha' y 'CodigoEmpleado')
maestra_resultados_por_empleado_centro_costo.dropna(subset=[
        'Fecha'
    ], inplace=True)
maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] = maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'].astype(str)
maestra_resultados_por_empleado_centro_costo.head()

In [ ]:
# TODO Hacer el merge con la tabla de promedio variable
maestra_resultados_por_empleado_centro_costo = maestra_resultados_por_empleado_centro_costo.merge(
    df_salarios_variables[['CodigoEmpleado', 'Fecha', 'PromedioSalarioVariable']].rename(
        columns={'PromedioSalarioVariable': 'SalarioVariable'}
    ),
    on=['CodigoEmpleado', 'Fecha'],
    how='left'
)
maestra_resultados_por_empleado_centro_costo.head()

In [ ]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

In [ ]:
maestra_resultados_por_empleado_centro_costo[maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '1032450706']

### Calculo sobre maestra resultados por empleado

In [ ]:
maestra_resultados_por_empleado = maestra_resultados_por_empleado.merge(
    df_factores_liquidacion,
    on=['TipoEmpleado', 'PorcentajeCumplimiento'],
    how='left'
)

maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
    ((maestra_resultados_por_empleado['Fecha'] >= '2022-08-01') & 
     (maestra_resultados_por_empleado['PorcentajeCumplimiento'] == 0) & 
     (maestra_resultados_por_empleado['Presupuesto'] == 0) & 
     (maestra_resultados_por_empleado['Real'] == 0)),
    0.25,
    maestra_resultados_por_empleado['FactorIncentivo']
)
maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
    ((maestra_resultados_por_empleado['Fecha'] >= '2022-08-01') & 
     (maestra_resultados_por_empleado['PorcentajeCumplimiento'] == 0) & 
     (maestra_resultados_por_empleado['Presupuesto'] == 0) & 
     (maestra_resultados_por_empleado['Real'] > 0)),
    0.5,
    maestra_resultados_por_empleado['FactorIncentivo']
)

##
_excepciones_porcentaje_cumplimiento = [
    # {
    #     'TipoEmpleado': 'VC',
    #     'AreaCalculo': 7,
    #     'Variable': 'VentaFacturadaZonaCargoIndividualSub',
    #     'Consecutivo': 4,
    #     'FechaInicial': '2021-02-01',
    #     'FechaFinal': '2021-06-01',
    #     'PorcentajeCumplimiento': 1,
    #     'FactorIncentivo': 1,
    #     'Codigos': ['5003527']
    # }
]

for excepcion in _excepciones_porcentaje_cumplimiento:
    maestra_resultados_por_empleado['PorcentajeCumplimiento'] = np.where(
        (
            (maestra_resultados_por_empleado['CodigoEmpleado'].isin(excepcion['Codigos'])) &
            (maestra_resultados_por_empleado['Consecutivo'] == excepcion['Consecutivo']) &
            (maestra_resultados_por_empleado['Variable'] == excepcion['Variable']) &
            (maestra_resultados_por_empleado['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado['Fecha'] <= excepcion['FechaFinal'])
        ),
        excepcion['PorcentajeCumplimiento'],
        maestra_resultados_por_empleado['PorcentajeCumplimiento']
    )
    maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
        (
            (maestra_resultados_por_empleado['CodigoEmpleado'].isin(excepcion['Codigos'])) &
            (maestra_resultados_por_empleado['Consecutivo'] == excepcion['Consecutivo']) &
            (maestra_resultados_por_empleado['Variable'] == excepcion['Variable']) &
            (maestra_resultados_por_empleado['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado['Fecha'] <= excepcion['FechaFinal'])
        ),
        excepcion['FactorIncentivo'],
        maestra_resultados_por_empleado['FactorIncentivo']
    )
##

maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
    (maestra_resultados_por_empleado['PorcentajeCumplimiento']<0.9) & (maestra_resultados_por_empleado['FactorIncentivo'].isnull()),
    0,
    maestra_resultados_por_empleado['FactorIncentivo']
)
maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
    (maestra_resultados_por_empleado['PorcentajeCumplimiento']>1.06) & (maestra_resultados_por_empleado['FactorIncentivo'].isnull()),
    2.5,
    maestra_resultados_por_empleado['FactorIncentivo']
)
maestra_resultados_por_empleado = maestra_resultados_por_empleado[maestra_resultados_por_empleado['Fecha'].notnull()]
maestra_resultados_por_empleado['FactorMes'] = pd.DatetimeIndex(maestra_resultados_por_empleado['Fecha']).month
maestra_resultados_por_empleado['FactorMes'] = maestra_resultados_por_empleado['FactorMes'].astype('int')
maestra_resultados_por_empleado['Liquidado'] = maestra_resultados_por_empleado['SalarioVariable']*maestra_resultados_por_empleado['Porcentaje']
maestra_resultados_por_empleado['Liquidado'] = maestra_resultados_por_empleado['Liquidado']*maestra_resultados_por_empleado['FactorIncentivo']

try:
    maestra_resultados_por_empleado['DifMonths'] = ((maestra_resultados_por_empleado.Fecha - maestra_resultados_por_empleado.FechaIngreso)/np.timedelta64(1, 'M'))
except:
    maestra_resultados_por_empleado['DifMonths'] = 0
maestra_resultados_por_empleado['DifMonths'].fillna(0, inplace=True)
maestra_resultados_por_empleado['DifMonths'] = maestra_resultados_por_empleado['DifMonths'].apply(lambda x: round(x, 0))
maestra_resultados_por_empleado['DifMonths'] = maestra_resultados_por_empleado['DifMonths'].astype('int')

maestra_resultados_por_empleado['DifMonths'] = maestra_resultados_por_empleado.groupby(['CodigoEmpleado'])['DifMonths'].transform(min)
maestra_resultados_por_empleado['DifMonths'] = np.where(
    maestra_resultados_por_empleado['DifMonths']>0, 0, maestra_resultados_por_empleado['DifMonths']
)
maestra_resultados_por_empleado['NuevoFactor'] = maestra_resultados_por_empleado['FactorMes'] + maestra_resultados_por_empleado['DifMonths']
maestra_resultados_por_empleado['NuevoFactor'] = maestra_resultados_por_empleado['NuevoFactor'].apply(lambda x: 0 if x<0 else x)

###
_excepciones_factor_mes = [
    # Garantizados para VEs y VCs
    {
        'TipoEmpleado': ['VC', 'VE'],
        'FechaInicial': '2022-03-01',
        'FechaFinal': '2022-12-01',
        'SumFactorMes': -2
    }
]
for excepcion in _excepciones_factor_mes:
    maestra_resultados_por_empleado['NuevoFactor'] = np.where(
        (
            (maestra_resultados_por_empleado['TipoEmpleado'].isin(excepcion['TipoEmpleado'])) &
            (maestra_resultados_por_empleado['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado['Fecha'] <= excepcion['FechaFinal'])
        ),
        maestra_resultados_por_empleado['NuevoFactor'] + excepcion['SumFactorMes'],
        maestra_resultados_por_empleado['NuevoFactor']
    )

maestra_resultados_por_empleado['NuevoFactor'] = np.where(
    maestra_resultados_por_empleado['NuevoFactor'] < 1,
    1,
    maestra_resultados_por_empleado['NuevoFactor']
)
maestra_resultados_por_empleado['NuevoFactorBK'] = maestra_resultados_por_empleado['NuevoFactor']
###

maestra_resultados_por_empleado['Liquidado'] = maestra_resultados_por_empleado['Liquidado']*maestra_resultados_por_empleado['NuevoFactor']

maestra_resultados_por_empleado = maestra_resultados_por_empleado[maestra_resultados_por_empleado['CodigoEmpleado'].notnull()]

In [ ]:
maestra_resultados_por_empleado

In [ ]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

### Calculo sobre maestra resultados por empleado centro costo

In [ ]:
maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo.merge(
    df_factores_liquidacion,
    on=['TipoEmpleado', 'PorcentajeCumplimiento'],
    how='left'
)

maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
    ((maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= '2022-08-01') & 
     (maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento'] == 0) & 
     (maestra_resultados_por_empleado_centro_costo_final['Presupuesto'] == 0) & 
     (maestra_resultados_por_empleado_centro_costo_final['Real'] == 0)),
    0.25,
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
)
maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
    ((maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= '2022-08-01') & 
     (maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento'] == 0) & 
     (maestra_resultados_por_empleado_centro_costo_final['Presupuesto'] == 0) & 
     (maestra_resultados_por_empleado_centro_costo_final['Real'] > 0)),
    0.5,
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
)

##
_excepciones_porcentaje_cumplimiento = [
    # {
    #     'TipoEmpleado': 'VC',
    #     'AreaCalculo': 7,
    #     'Variable': 'VentaFacturadaZonaCargoIndividualSub',
    #     'Consecutivo': 4,
    #     'FechaInicial': '2021-02-01',
    #     'FechaFinal': '2021-06-01',
    #     'PorcentajeCumplimiento': 1,
    #     'FactorIncentivo': 1,
    #     'Codigos': ['5003527']
    # }
]

for excepcion in _excepciones_porcentaje_cumplimiento:
    maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento'] = np.where(
        (
            (maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'].isin(excepcion['Codigos'])) &
            (maestra_resultados_por_empleado_centro_costo_final['Consecutivo'] == excepcion['Consecutivo']) &
            (maestra_resultados_por_empleado_centro_costo_final['Variable'] == excepcion['Variable']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] <= excepcion['FechaFinal'])
        ),
        excepcion['PorcentajeCumplimiento'],
        maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento']
    )
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
        (
            (maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'].isin(excepcion['Codigos'])) &
            (maestra_resultados_por_empleado_centro_costo_final['Consecutivo'] == excepcion['Consecutivo']) &
            (maestra_resultados_por_empleado_centro_costo_final['Variable'] == excepcion['Variable']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] <= excepcion['FechaFinal'])
        ),
        excepcion['FactorIncentivo'],
        maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
    )
##

maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
    (maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento']<0.9) & (maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'].isnull()),
    0,
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
)
maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
    (maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento']>1.06) & (maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'].isnull()),
    2.5,
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
)
maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final[maestra_resultados_por_empleado_centro_costo_final['Fecha'].notnull()]
maestra_resultados_por_empleado_centro_costo_final['FactorMes'] = pd.DatetimeIndex(maestra_resultados_por_empleado_centro_costo_final['Fecha']).month
maestra_resultados_por_empleado_centro_costo_final['FactorMes'] = maestra_resultados_por_empleado_centro_costo_final['FactorMes'].astype('int')
maestra_resultados_por_empleado_centro_costo_final['Liquidado'] = maestra_resultados_por_empleado_centro_costo_final['SalarioVariable']*maestra_resultados_por_empleado_centro_costo_final['Porcentaje']
maestra_resultados_por_empleado_centro_costo_final['Liquidado'] = maestra_resultados_por_empleado_centro_costo_final['Liquidado']*maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']

try:
    maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = ((maestra_resultados_por_empleado_centro_costo_final.Fecha - maestra_resultados_por_empleado_centro_costo_final.FechaIngreso)/np.timedelta64(1, 'M'))
except:
    maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = 0
maestra_resultados_por_empleado_centro_costo_final['DifMonths'].fillna(0, inplace=True)
maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = maestra_resultados_por_empleado_centro_costo_final['DifMonths'].apply(lambda x: round(x, 0))
maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = maestra_resultados_por_empleado_centro_costo_final['DifMonths'].astype('int')

maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = maestra_resultados_por_empleado_centro_costo_final.groupby(['CodigoEmpleado'])['DifMonths'].transform(min)
maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = np.where(
    maestra_resultados_por_empleado_centro_costo_final['DifMonths']>0, 0, maestra_resultados_por_empleado_centro_costo_final['DifMonths']
)
maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] = maestra_resultados_por_empleado_centro_costo_final['FactorMes'] + maestra_resultados_por_empleado_centro_costo_final['DifMonths']
maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] = maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'].apply(lambda x: 0 if x<0 else x)

###
_excepciones_factor_mes = [
    # Garantizados para VEs y VCs
    {
        'TipoEmpleado': ['VC', 'VE'],
        'FechaInicial': '2022-03-01',
        'FechaFinal': '2022-12-01',
        'SumFactorMes': -2
    }
]
for excepcion in _excepciones_factor_mes:
    maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] = np.where(
        (
            (maestra_resultados_por_empleado_centro_costo_final['TipoEmpleado'].isin(excepcion['TipoEmpleado'])) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] <= excepcion['FechaFinal'])
        ),
        maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] + excepcion['SumFactorMes'],
        maestra_resultados_por_empleado_centro_costo_final['NuevoFactor']
    )

maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] = np.where(
    maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] < 1,
    1,
    maestra_resultados_por_empleado_centro_costo_final['NuevoFactor']
)
maestra_resultados_por_empleado_centro_costo_final['NuevoFactorBK'] = maestra_resultados_por_empleado_centro_costo_final['NuevoFactor']
###

maestra_resultados_por_empleado_centro_costo_final['Liquidado'] = maestra_resultados_por_empleado_centro_costo_final['Liquidado']*maestra_resultados_por_empleado_centro_costo_final['NuevoFactor']

maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final[maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'].notnull()]

In [ ]:
maestra_resultados_por_empleado_centro_costo_final.head()

In [ ]:
maestra_resultados_por_empleado_centro_costo_final[maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'] == '1098612757']

## Se calculan los resultados por mes

In [ ]:
lista_variables_rentabilidad_acumuladas = [
    'RentabilidadBbraunPaisAntesDeImpuestos',
    'RentabilidadDivisionGrupoClientesCM3',
    'RentabilidadDivisionGrupoClientesCM3Sub',
    'RentabilidadUnidadNegocioCM2',
    'RentabilidadUnidadNegocioCM2Sub',
    'RentabilidadUnidadNegocioCM3',
    'RentabilidadUnidadNegocioCM3Sub',
    'TotalRentabilidadBraunAntesDeImpuestosCM5',
    'VentaFacturadaPrevioNotasCreditoTP',
    # 2022
    'RentabilidadCompaniaCM3',
    'RentabilidadZonaHospitalarioCM2',
    # 2024
    'RentabilidadUnidadNegocioCM2COGSSUB',
    'RentabilidadDivisionCM2COGsSub',
    'EBITCOGSPais',
    'EBITCOGSPaisSUB'
]

def calcular_resultado_mes(group):
    valor_previo_real = 0
    valor_previo_presupuesto = 0
    for index, item in group.iterrows():
        if item['Variable'] in lista_variables_rentabilidad_acumuladas:
            resultado_real_mes = item['Real']
            resultado_presupuesto_mes = item['Presupuesto']
        else:
            resultado_real_mes = item['Real'] - valor_previo_real
            resultado_presupuesto_mes = item['Presupuesto'] - valor_previo_presupuesto
        
        group.at[index, 'ResultadoRealMes'] = resultado_real_mes
        group.at[index, 'ResultadoPresupuesto'] = resultado_presupuesto_mes

        porcentaje_cumplimiento_mes = 0
        if resultado_presupuesto_mes == 0:
            porcentaje_cumplimiento_mes = 0
        elif resultado_real_mes - resultado_presupuesto_mes < 0:
            porcentaje_cumplimiento_mes = 1 + ((resultado_real_mes - resultado_presupuesto_mes) / abs(resultado_presupuesto_mes))
        elif resultado_real_mes - resultado_presupuesto_mes > 0:
            porcentaje_cumplimiento_mes = 1 + abs((resultado_real_mes - resultado_presupuesto_mes) / abs(resultado_presupuesto_mes))
        else:
            porcentaje_cumplimiento_mes = resultado_real_mes / resultado_presupuesto_mes

        group.at[index, 'PorcentajeCumplimientoMes'] = round(porcentaje_cumplimiento_mes, 2)
        
        valor_previo_real = item['Real']
        valor_previo_presupuesto = item['Presupuesto']
    return group


### Maestra Resultados por Empleado vs Resultado Detallado Histórico

In [ ]:
maestra_resultados_por_empleado['Real'].fillna(0,inplace=True)
maestra_resultados_por_empleado['Presupuesto'].fillna(0,inplace=True)
maestra_resultados_por_empleado['ResultadoPresupuesto'] = 0
maestra_resultados_por_empleado['ResultadoRealMes'] = 0
maestra_resultados_por_empleado['PorcentajeCumplimientoMes'] = 0
maestra_resultados_por_empleado['PorcentajeCumplimientoMes'] = maestra_resultados_por_empleado['PorcentajeCumplimientoMes'].astype('float')
maestra_resultados_por_empleado = maestra_resultados_por_empleado.groupby(['CodigoEmpleado', 'Variable', 'Contexto'], group_keys=False).apply(calcular_resultado_mes)

maestra_resultados_por_empleado

In [ ]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

In [ ]:
df_resultado_detallado_previo

In [ ]:
df_resultado_detallado_previo[
    (df_resultado_detallado_previo['Contexto']=='1111117') &
    (df_resultado_detallado_previo['Variable']=='ObjetivosCualitativos')
]

In [ ]:
# Se une el resultadio histórico con el actual, de tal forma que al histórico se le agregan únicamenet 
# los resultados correspondientes al mes que se está liquidando
resultado_detallado_historico = df_resultado_detallado_previo.copy()
# Filtrar el DataFrame para encontrar las filas donde 'CodigoEmpleado' termina con '.0'
resultado_detallado_historico[resultado_detallado_historico['CodigoEmpleado'].astype(str).str.endswith('.0')].head()
resultado_detallado_historico['CodigoEmpleado'] = resultado_detallado_historico['CodigoEmpleado'].astype(str).str.replace(r'(\d+)\.0$', r'\1', regex=True)
resultado_detallado_historico


In [ ]:
# resultado_detallado_historico.info()
# resultado_detallado_historico.to_excel('resultado_detallado_historico.xlsx')

In [ ]:
fecha_liquidacion

In [ ]:
last_date_historico = resultado_detallado_historico['Fecha'].max()
last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico
last_date_historico

In [ ]:
# maestra_resultados_por_empleado[
#     maestra_resultados_por_empleado['Fecha']>last_date_historico
# ]

In [ ]:
resultado_detallado_historico[
    (resultado_detallado_historico['Contexto']=='1111117') &
    (resultado_detallado_historico['Variable']=='ObjetivosCualitativos')
]

In [ ]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

In [ ]:
maestra_resultados_por_empleado = maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Fecha']<=fecha_liquidacion)
]

maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

In [ ]:
if last_date_historico:
    maestra_resultados_por_empleado = maestra_resultados_por_empleado[
        maestra_resultados_por_empleado['Fecha']>last_date_historico
    ]

In [ ]:
# last_date_historico = resultado_detallado_historico['Fecha'].max()
# last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico

try:
    maestra_resultados_por_empleado = maestra_resultados_por_empleado[
        (maestra_resultados_por_empleado['Fecha']<=fecha_liquidacion)
    ]
    if last_date_historico:
        maestra_resultados_por_empleado = maestra_resultados_por_empleado[
            maestra_resultados_por_empleado['Fecha']>last_date_historico
        ]
except:
    raise

In [ ]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

In [ ]:
maestra_resultados_por_empleado = pd.concat([resultado_detallado_historico, maestra_resultados_por_empleado])
maestra_resultados_por_empleado = maestra_resultados_por_empleado.sort_values(by=['CodigoEmpleado', 'Fecha'])
maestra_resultados_por_empleado

In [ ]:
maestra_resultados_por_empleado['Variable'].drop_duplicates()

In [ ]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

### Maestra Resultados por Empleado Centro Costo vs Resultado Detallado Histórico

In [ ]:
maestra_resultados_por_empleado_centro_costo_final['Real'].fillna(0,inplace=True)
maestra_resultados_por_empleado_centro_costo_final['Presupuesto'].fillna(0,inplace=True)
maestra_resultados_por_empleado_centro_costo_final['ResultadoPresupuesto'] = 0
maestra_resultados_por_empleado_centro_costo_final['ResultadoRealMes'] = 0
maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimientoMes'] = 0
maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimientoMes'] = maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimientoMes'].astype('float')
maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final.groupby(['CodigoEmpleado', 'Variable', 'Contexto'], group_keys=False).apply(calcular_resultado_mes)

maestra_resultados_por_empleado_centro_costo_final.head()

In [ ]:
df_resultado_detallado_previo.copy().merge(
    df_area_calculo_sba_centros_costos.rename(columns={'ConsecutivoParrilla':'Consecutivo'}),
    on=['AreaCalculo', 'Consecutivo','TipoEmpleado'],
    how='left'    
)

# Se une el resultadio histórico con el actual, de tal forma que al histórico se le agregan únicamenet 
# los resultados correspondientes al mes que se está liquidando
resultado_detallado_historico = df_resultado_detallado_previo.copy()
last_date_historico = resultado_detallado_historico['Fecha'].max()
last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico

try:
    maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final[
        (maestra_resultados_por_empleado_centro_costo_final['Fecha']<=fecha_liquidacion)
    ]
    if last_date_historico:
        maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final[
            maestra_resultados_por_empleado_centro_costo_final['Fecha']>last_date_historico
        ]
except:
    raise

maestra_resultados_por_empleado_centro_costo_final = pd.concat([resultado_detallado_historico, maestra_resultados_por_empleado_centro_costo_final])
maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final.sort_values(by=['CodigoEmpleado', 'Fecha'])
maestra_resultados_por_empleado_centro_costo_final

In [ ]:
maestra_resultados_por_empleado_centro_costo_final[maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'] == '1098612757']


In [ ]:
maestra_resultados_por_empleado[maestra_resultados_por_empleado['Variable'] == 'VentaFacturadaGrupoClientesKAMSSub']

In [ ]:
maestra_resultados_por_empleado.to_excel('resultados_detallados.xlsx')

In [ ]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['CodigoEmpleado'] == '1111117') &
     (maestra_resultados_por_empleado['Variable'] == 'ObjetivosCualitativos')
]

In [ ]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['TipoEmpleado'] == 'VC') |
    (maestra_resultados_por_empleado['TipoEmpleado'] == 'VE')
]['Variable'].drop_duplicates()

In [ ]:
maestra_resultados_por_empleado[maestra_resultados_por_empleado['Variable'] == 'RentabilidadUnidadNegocioCM2COGS']

In [ ]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Variable'] == 'VentaFacturadaPortafolioRenalSub')
]

## Se obtienen las liquidaciones

### Creación de resultado_liquidaciones por maestra_resultados_por_empleado

In [ ]:
excepciones_pagos_acumulados = {}

excepciones_diferencias = {}

In [ ]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['CodigoEmpleado'] == '16659046') &
    (maestra_resultados_por_empleado['Contexto'] == '16659046') 
#     (maestra_resultados_por_empleado['Fecha'] == '2024-04-01')
][['Contexto', 'CodigoEmpleado', 'SalarioVariable', 'Fecha', 'NuevoFactorBK', 'Liquidado']]

In [ ]:
df_incentivos_por_empleado[df_incentivos_por_empleado['CodigoEmpleado'] == '16659046']

In [ ]:
resultado_liquidaciones = maestra_resultados_por_empleado.groupby(
    ['CodigoEmpleado', 'SalarioVariable', 'Fecha', 'NuevoFactorBK'],
    as_index=False
).Liquidado.agg({'Liquidado': 'sum'})
resultado_liquidaciones.rename(columns={'NuevoFactorBK': 'NuevoFactor'}, inplace=True)
resultado_liquidaciones.head()
resultado_liquidaciones = resultado_liquidaciones.merge(
    df_incentivos_por_empleado.rename(columns={'Valor': 'PagoReal'}),
    on = ['CodigoEmpleado', 'Fecha'],
    how = 'right'
)
resultado_liquidaciones.head()

In [ ]:
resultado_liquidaciones[resultado_liquidaciones['CodigoEmpleado'] == '1010166177']

In [ ]:
resultado_liquidaciones[
    (resultado_liquidaciones['CodigoEmpleado'] == '16659046')
]

In [ ]:
resultado_liquidaciones = pd.merge(
    left=resultado_liquidaciones,
    right=df_empleados[['CodigoEmpleado', 'FechaIngreso']].drop_duplicates(['CodigoEmpleado']),
    how='left',
    on=['CodigoEmpleado']
)
resultado_liquidaciones = resultado_liquidaciones[resultado_liquidaciones['Fecha'] >= resultado_liquidaciones['FechaIngreso']]
resultado_liquidaciones.drop(columns=['FechaIngreso'], inplace=True)
resultado_liquidaciones.head()

# Cálculo de garantizado (100%) para VEs y VCs, Anticipos (85%) para el resto de empleados
resultado_liquidaciones = resultado_liquidaciones.merge(
    df_empleados[['CodigoEmpleado', 'TipoEmpleado']],
    on='CodigoEmpleado',
    how='left'
)

def calcular_liquidacion_mes(g):
    pago_acumulado = 0
    for i, row in g.iterrows():
        print("mes actual:", row['Fecha'].month)
        if row['Fecha'].month in meses_incentivos:
            print("Pago Real", g.at[i, 'PagoReal'])
            if g.at[i, 'PagoReal'] == 0:
                print("Pago Real es 0", g.at[i, 'PagoReal'])
                if i == 0:
                    print("i es igual a 0")
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado']
                else:
                    print("i no es igual a 0")
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado'] - pago_acumulado
                print("Pago Real actualizado", g.at[i, 'PagoReal'])
            if g.at[i, 'PagoReal'] < 0:
                g.at[i, 'PagoReal'] = 0.0
        print("pago_acumuladol_antes:", pago_acumulado)
        pago_acumulado += g.at[i, 'PagoReal']
        print("pago_acumuladol_después:", pago_acumulado)
        
        excepcion = excepciones_pagos_acumulados.get(row['CodigoEmpleado'], None)
        if excepcion and row['Fecha'] == excepcion['Fecha']:
            pago_acumulado = excepcion['ValorPagoAcumulado']
        
        excepcion_diferencia = excepciones_diferencias.get(row['CodigoEmpleado'], None)
        if excepcion_diferencia and row['Fecha'] == excepcion_diferencia['Fecha']:
            pago_acumulado = pago_acumulado + excepcion_diferencia['ValorDiferencia']
        
    return g
resultado_liquidaciones = resultado_liquidaciones.sort_values(by=['CodigoEmpleado', 'Fecha'])
resultado_liquidaciones.reset_index(inplace=True, drop=True)
resultado_liquidaciones['PagoReal'].fillna(0.0, inplace=True)
resultado_liquidaciones['Liquidado'].fillna(0.0, inplace=True)
resultado_liquidaciones

In [ ]:
resultado_liquidaciones[resultado_liquidaciones['CodigoEmpleado'] == '16659046']

In [ ]:
resultado_liquidaciones[
    (resultado_liquidaciones['CodigoEmpleado'] == '16659046')
]

In [ ]:
resultado_liquidaciones_copy = resultado_liquidaciones[
    (resultado_liquidaciones['CodigoEmpleado'] == '16659046')
].groupby(['CodigoEmpleado'], group_keys=False).apply(calcular_liquidacion_mes)
resultado_liquidaciones_copy[resultado_liquidaciones_copy['CodigoEmpleado'] == '16659046']

In [ ]:
resultado_liquidaciones = resultado_liquidaciones.groupby(['CodigoEmpleado'], group_keys=False).apply(calcular_liquidacion_mes)
resultado_liquidaciones.rename(columns={'NuevoFactor': 'FactorMes'}, inplace=True)
resultado_liquidaciones['PagoGarantizado'] = 'NO'

resultado_liquidaciones['PagoReal'] = np.where(
    (
        (resultado_liquidaciones['Fecha'] >= '2022-01-01') &
        (resultado_liquidaciones['Fecha'] <= '2022-02-01') &
        (resultado_liquidaciones['TipoEmpleado'].isin(['VE', 'VC']))
    ),
    resultado_liquidaciones['SalarioVariable'],
    resultado_liquidaciones['PagoReal']
)
resultado_liquidaciones['PagoReal'] = np.where(
    (
        (resultado_liquidaciones['Fecha'] >= '2022-01-01') &
        (resultado_liquidaciones['Fecha'] <= '2022-02-01') &
        (resultado_liquidaciones['TipoEmpleado'].isin(['AD', 'AD1', 'AD2', 'AD3', 'AD4', 'AD5', 'GC', 'GD', 'GU', 'KAM', 'MK']))
    ),
    resultado_liquidaciones['SalarioVariable'] * 0.85,
    resultado_liquidaciones['PagoReal']
)
resultado_liquidaciones.drop(columns=['TipoEmpleado'], inplace=True)

resultado_liquidaciones.head(12)

In [ ]:
resultado_liquidaciones[resultado_liquidaciones['CodigoEmpleado'] == '16659046']

### Creación de resultado_liquidaciones por maestra_resultados_por_empleado_centro_costo

In [ ]:
resultado_liquidaciones_centro_costo = maestra_resultados_por_empleado_centro_costo_final.groupby(
    ['CodigoEmpleado', 'CodigoCentroCosto','SalarioVariable', 'Fecha', 'NuevoFactorBK'],
    as_index=False
).Liquidado.agg({'Liquidado': 'sum'})
resultado_liquidaciones_centro_costo.rename(columns={'NuevoFactorBK': 'NuevoFactor'}, inplace=True)
resultado_liquidaciones_centro_costo.head()
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.merge(
    df_incentivos_por_empleado.rename(columns={'Valor': 'PagoReal'}),
    on = ['CodigoEmpleado', 'Fecha'],
    how = 'right'
)

resultado_liquidaciones_centro_costo = pd.merge(
    left=resultado_liquidaciones_centro_costo,
    right=df_empleados[['CodigoEmpleado', 'FechaIngreso']].drop_duplicates(['CodigoEmpleado']),
    how='left',
    on=['CodigoEmpleado']
)

resultado_liquidaciones_centro_costo.head()

In [ ]:
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo[resultado_liquidaciones_centro_costo['Fecha'] >= resultado_liquidaciones_centro_costo['FechaIngreso']]
resultado_liquidaciones_centro_costo.drop(columns=['FechaIngreso'], inplace=True)
resultado_liquidaciones_centro_costo.head()

# Cálculo de garantizado (100%) para VEs y VCs, Anticipos (85%) para el resto de empleados
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.merge(
    df_empleados[['CodigoEmpleado', 'TipoEmpleado']],
    on='CodigoEmpleado',
    how='left'
)

def calcular_liquidacion_mes(g):
    pago_acumulado = 0
    for i, row in g.iterrows():
        if row['Fecha'].month in meses_incentivos:
            if g.at[i, 'PagoReal'] == 0:
                if i == 0:
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado']
                else:
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado'] - pago_acumulado
            
            if g.at[i, 'PagoReal'] < 0:
                g.at[i, 'PagoReal'] = 0.0
        pago_acumulado += g.at[i, 'PagoReal']
        
        excepcion = excepciones_pagos_acumulados.get(row['CodigoEmpleado'], None)
        if excepcion and row['Fecha'] == excepcion['Fecha']:
            pago_acumulado = excepcion['ValorPagoAcumulado']
        
        excepcion_diferencia = excepciones_diferencias.get(row['CodigoEmpleado'], None)
        if excepcion_diferencia and row['Fecha'] == excepcion_diferencia['Fecha']:
            pago_acumulado = pago_acumulado + excepcion_diferencia['ValorDiferencia']
        
    return g
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.sort_values(by=['CodigoEmpleado', 'Fecha'])
resultado_liquidaciones_centro_costo.reset_index(inplace=True, drop=True)
resultado_liquidaciones_centro_costo['PagoReal'].fillna(0.0, inplace=True)
resultado_liquidaciones_centro_costo['Liquidado'].fillna(0.0, inplace=True)
resultado_liquidaciones_centro_costo

In [ ]:
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.groupby(['CodigoEmpleado'], group_keys=False).apply(calcular_liquidacion_mes)
resultado_liquidaciones_centro_costo.rename(columns={'NuevoFactor': 'FactorMes'}, inplace=True)
resultado_liquidaciones_centro_costo['PagoGarantizado'] = 'NO'

resultado_liquidaciones_centro_costo['PagoReal'] = np.where(
    (
        (resultado_liquidaciones_centro_costo['Fecha'] >= '2022-01-01') &
        (resultado_liquidaciones_centro_costo['Fecha'] <= '2022-02-01') &
        (resultado_liquidaciones_centro_costo['TipoEmpleado'].isin(['VE', 'VC']))
    ),
    resultado_liquidaciones_centro_costo['SalarioVariable'],
    resultado_liquidaciones_centro_costo['PagoReal']
)
resultado_liquidaciones_centro_costo['PagoReal'] = np.where(
    (
        (resultado_liquidaciones_centro_costo['Fecha'] >= '2022-01-01') &
        (resultado_liquidaciones_centro_costo['Fecha'] <= '2022-02-01') &
        (resultado_liquidaciones_centro_costo['TipoEmpleado'].isin(['AD', 'AD1', 'AD2', 'AD3', 'AD4', 'AD5', 'GC', 'GD', 'GU', 'KAM', 'MK']))
    ),
    resultado_liquidaciones_centro_costo['SalarioVariable'] * 0.85,
    resultado_liquidaciones_centro_costo['PagoReal']
)
resultado_liquidaciones_centro_costo.drop(columns=['TipoEmpleado'], inplace=True)

resultado_liquidaciones_centro_costo.head(12)

## Se obtienen los garantizados

### Sin centro de costo

In [ ]:
empleados_garantizado = df_empleados[df_empleados['FechaIngreso'] > fecha_liquidacion][['CodigoEmpleado', 'SalarioVariable', 'FechaIngreso']]
fechas_resultado_liquidaciones = resultado_liquidaciones[['Fecha']].drop_duplicates()

empleados_garantizado['Key'] = 1
fechas_resultado_liquidaciones['Key'] = 1

liquidaciones_garantizados = pd.merge(
    left=fechas_resultado_liquidaciones.sort_values(['Fecha']),
    right=empleados_garantizado,
    how='outer',
    on=['Key']
).drop(columns=['Key'])

liquidaciones_garantizados = liquidaciones_garantizados[liquidaciones_garantizados['CodigoEmpleado'].notnull()]

liquidaciones_garantizados['DifMonths'] = ((liquidaciones_garantizados.FechaIngreso - liquidaciones_garantizados.Fecha)/np.timedelta64(1, 'M'))
liquidaciones_garantizados['DifMonths'] = liquidaciones_garantizados['DifMonths'].apply(lambda x: round(x, 0))
liquidaciones_garantizados['DifMonths'] = liquidaciones_garantizados['DifMonths'].astype('int')
liquidaciones_garantizados = liquidaciones_garantizados[liquidaciones_garantizados['DifMonths'] <= 4]

liquidaciones_garantizados['FactorMes'] = 0
liquidaciones_garantizados['Liquidado'] = liquidaciones_garantizados['SalarioVariable']
liquidaciones_garantizados['PagoReal'] = liquidaciones_garantizados['SalarioVariable']
liquidaciones_garantizados['PagoGarantizado'] = 'SI'

liquidaciones_garantizados.drop(columns=['DifMonths', 'FechaIngreso'], inplace=True)

liquidaciones_garantizados.head()

### Con centro de costo

In [ ]:
empleados_garantizado = df_empleados[df_empleados['FechaIngreso'] > fecha_liquidacion][['CodigoEmpleado', 'SalarioVariable', 'FechaIngreso']]
fechas_resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo[['Fecha']].drop_duplicates()

empleados_garantizado['Key'] = 1
fechas_resultado_liquidaciones_centro_costo['Key'] = 1

liquidaciones_garantizados_centro_costo = pd.merge(
    left=fechas_resultado_liquidaciones_centro_costo.sort_values(['Fecha']),
    right=empleados_garantizado,
    how='outer',
    on=['Key']
).drop(columns=['Key'])

liquidaciones_garantizados_centro_costo = liquidaciones_garantizados_centro_costo[liquidaciones_garantizados_centro_costo['CodigoEmpleado'].notnull()]

liquidaciones_garantizados_centro_costo['DifMonths'] = ((liquidaciones_garantizados_centro_costo.FechaIngreso - liquidaciones_garantizados_centro_costo.Fecha)/np.timedelta64(1, 'M'))
liquidaciones_garantizados_centro_costo['DifMonths'] = liquidaciones_garantizados_centro_costo['DifMonths'].apply(lambda x: round(x, 0))
liquidaciones_garantizados_centro_costo['DifMonths'] = liquidaciones_garantizados_centro_costo['DifMonths'].astype('int')
liquidaciones_garantizados_centro_costo = liquidaciones_garantizados_centro_costo[liquidaciones_garantizados_centro_costo['DifMonths'] <= 4]

liquidaciones_garantizados_centro_costo['FactorMes'] = 0
liquidaciones_garantizados_centro_costo['Liquidado'] = liquidaciones_garantizados_centro_costo['SalarioVariable']
liquidaciones_garantizados_centro_costo['PagoReal'] = liquidaciones_garantizados_centro_costo['SalarioVariable']
liquidaciones_garantizados_centro_costo['PagoGarantizado'] = 'SI'

liquidaciones_garantizados_centro_costo.drop(columns=['DifMonths', 'FechaIngreso'], inplace=True)

liquidaciones_garantizados_centro_costo.head()

## Se unen las liquidaciones de garantizados con las demás

### Sin centro de costo

In [ ]:
resultado_liquidaciones = pd.concat([resultado_liquidaciones, liquidaciones_garantizados])
resultado_liquidaciones.reset_index(drop=True, inplace=True)
resultado_liquidaciones

### Con centro de costo

In [ ]:
resultado_liquidaciones_centro_costo = pd.concat([resultado_liquidaciones_centro_costo, liquidaciones_garantizados_centro_costo])
resultado_liquidaciones_centro_costo.reset_index(drop=True, inplace=True)
resultado_liquidaciones_centro_costo

## Se conserva la información del histórico y se le añade la nueva información hasta la fecha de liquidación

### Sin centro de costo

In [ ]:
resultado_liquidaciones_historico = df_resultado_liquidaciones_previo.copy()
resultado_liquidaciones_historico['PagoGarantizado'] = resultado_liquidaciones_historico['PagoGarantizado'].fillna('NO')
last_date_historico = resultado_liquidaciones_historico['Fecha'].max()
last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico

try:
    resultado_liquidaciones = resultado_liquidaciones[
        resultado_liquidaciones['Fecha']<=fecha_liquidacion
    ]
    if last_date_historico:
        resultado_liquidaciones = resultado_liquidaciones[
            resultado_liquidaciones['Fecha']>last_date_historico
        ]
except:
    pass

resultado_liquidaciones = pd.concat([resultado_liquidaciones_historico, resultado_liquidaciones])
resultado_liquidaciones = resultado_liquidaciones.sort_values(by=['CodigoEmpleado', 'Fecha'])

for codigo_empleado_excepcion in excepciones_pagos_acumulados:
    excepcion = excepciones_pagos_acumulados[codigo_empleado_excepcion]
    resultado_liquidaciones['PagoReal'] = np.where(
        (
            (resultado_liquidaciones['CodigoEmpleado'] == codigo_empleado_excepcion) &
            (resultado_liquidaciones['Fecha'] <= excepcion['Fecha'])
        ),
        excepcion['ValorPagoAcumulado'],
        resultado_liquidaciones['PagoReal']
    )

for codigo_empleado_excepcion in excepciones_diferencias:
    excepcion = excepciones_diferencias[codigo_empleado_excepcion]
    resultado_liquidaciones['PagoReal'] = np.where(
        (
            (resultado_liquidaciones['CodigoEmpleado'] == codigo_empleado_excepcion) &
            (resultado_liquidaciones['Fecha'] == excepcion['Fecha'])
        ),
        resultado_liquidaciones['PagoReal'] + excepcion['ValorDiferencia'],
        resultado_liquidaciones['PagoReal']
    )

resultado_liquidaciones['PagadoFecha'] = resultado_liquidaciones.groupby(['CodigoEmpleado'])['PagoReal'].transform(pd.Series.cumsum)
resultado_liquidaciones['PagadoFecha'] = resultado_liquidaciones['PagadoFecha'] - resultado_liquidaciones['PagoReal']
resultado_liquidaciones

In [ ]:
resultado_liquidaciones.drop_duplicates(['CodigoEmpleado', 'Fecha'], inplace=True)

resultado_final = pd.merge(
    left=resultado_liquidaciones[['CodigoEmpleado', 'Fecha', 'PagoGarantizado', 'Liquidado', 'PagoReal', 'PagadoFecha', 'FactorMes']],
    right=df_empleados,
    on=['CodigoEmpleado'],
    how='left'
)
resultado_final = resultado_final[['Identificacion', 'Nombre', 'Apellidos', 'CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Fecha', 'SalarioVariable', 'PagoGarantizado', 'FactorMes', 'Liquidado', 'PagoReal', 'PagadoFecha']]
resultado_final = resultado_final.sort_values(by=['TipoEmpleado', 'AreaCalculo', 'CodigoEmpleado', 'Fecha'])

resultado_final.rename(columns={
    'Identificacion': 'Cédula',
    'CodigoEmpleado': 'Código',
    'TipoEmpleado': 'Tipo Empleado',
    'AreaCalculo': 'Área de cálculo',
    'Liquidado': 'Valor liquidado',
    'PagoReal': 'Valor neto a pagar',
    'PagadoFecha': 'Pagado a la fecha',
    'SalarioVariable': 'Salario variable',
    'FactorMes': 'Factor mes',
    'PagoGarantizado': '¿Pago garantizado?'
}, inplace=True)
resultado_final.head()

### Con centro de costo

In [ ]:
resultado_liquidaciones_historico_centro_costo = df_resultado_liquidaciones_previo.copy()
resultado_liquidaciones_historico_centro_costo['PagoGarantizado'] = resultado_liquidaciones_historico_centro_costo['PagoGarantizado'].fillna('NO')
last_date_historico = resultado_liquidaciones_historico_centro_costo['Fecha'].max()
last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico

try:
    resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo[
        resultado_liquidaciones['Fecha']<=fecha_liquidacion
    ]
    if last_date_historico:
        resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo[
            resultado_liquidaciones_centro_costo['Fecha']>last_date_historico
        ]
except:
    pass

resultado_liquidaciones_centro_costo = pd.concat([resultado_liquidaciones_historico_centro_costo, resultado_liquidaciones_centro_costo])
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.sort_values(by=['CodigoEmpleado', 'Fecha'])

for codigo_empleado_excepcion in excepciones_pagos_acumulados:
    excepcion = excepciones_pagos_acumulados[codigo_empleado_excepcion]
    resultado_liquidaciones_centro_costo['PagoReal'] = np.where(
        (
            (resultado_liquidaciones_centro_costo['CodigoEmpleado'] == codigo_empleado_excepcion) &
            (resultado_liquidaciones_centro_costo['Fecha'] <= excepcion['Fecha'])
        ),
        excepcion['ValorPagoAcumulado'],
        resultado_liquidaciones_centro_costo['PagoReal']
    )

for codigo_empleado_excepcion in excepciones_diferencias:
    excepcion = excepciones_diferencias[codigo_empleado_excepcion]
    resultado_liquidaciones_centro_costo['PagoReal'] = np.where(
        (
            (resultado_liquidaciones_centro_costo['CodigoEmpleado'] == codigo_empleado_excepcion) &
            (resultado_liquidaciones_centro_costo['Fecha'] == excepcion['Fecha'])
        ),
        resultado_liquidaciones_centro_costo['PagoReal'] + excepcion['ValorDiferencia'],
        resultado_liquidaciones_centro_costo['PagoReal']
    )

resultado_liquidaciones_centro_costo['PagadoFecha'] = resultado_liquidaciones_centro_costo.groupby(['CodigoEmpleado'])['PagoReal'].transform(pd.Series.cumsum)
resultado_liquidaciones_centro_costo['PagadoFecha'] = resultado_liquidaciones_centro_costo['PagadoFecha'] - resultado_liquidaciones_centro_costo['PagoReal']
resultado_liquidaciones_centro_costo

In [ ]:
resultado_liquidaciones_centro_costo.drop_duplicates(['CodigoEmpleado', 'Fecha'], inplace=True)

resultado_final_centro_costo = pd.merge(
    left=resultado_liquidaciones_centro_costo[['CodigoEmpleado', 'Fecha', 'PagoGarantizado', 'Liquidado', 'PagoReal','PagadoFecha', 'FactorMes', 'CodigoCentroCosto']],
    right=df_empleados,
    on=['CodigoEmpleado'],
    how='left'
)
resultado_final_centro_costo = resultado_final_centro_costo[['Identificacion', 'Nombre', 'Apellidos', 'CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Fecha', 'SalarioVariable', 'PagoGarantizado', 'FactorMes', 'Liquidado', 'PagoReal', 'PagadoFecha', 'CodigoCentroCosto']]
resultado_final_centro_costo = resultado_final_centro_costo.sort_values(by=['TipoEmpleado', 'AreaCalculo', 'CodigoEmpleado', 'Fecha', 'CodigoCentroCosto'])

resultado_final_centro_costo.rename(columns={
    'Identificacion': 'Cédula',
    'CodigoEmpleado': 'Código',
    'TipoEmpleado': 'Tipo Empleado',
    'AreaCalculo': 'Área de cálculo',
    'Liquidado': 'Valor liquidado',
    'PagoReal': 'Valor neto a pagar',
    'PagadoFecha': 'Pagado a la fecha',
    'SalarioVariable': 'Salario variable',
    'FactorMes': 'Factor mes',
    'PagoGarantizado': '¿Pago garantizado?'
}, inplace=True)
resultado_final_centro_costo.head()

In [ ]:
resultado_liquidaciones[resultado_liquidaciones['CodigoEmpleado'] == '16659046']

In [ ]:
resultado_final_centro_costo.to_excel(os.path.join(OUTPUT_FOLDER, 'resultado_final_centro_costo.xlsx'))
maestra_resultados_por_empleado.to_excel(os.path.join(OUTPUT_FOLDER, 'maestra_resultados_por_empleado.xlsx'))
maestra_resultados_por_empleado_centro_costo.to_excel(os.path.join(OUTPUT_FOLDER, 'maestra_resultados_por_empleado_centro_costo.xlsx'))
resultado_liquidaciones.to_excel(os.path.join(OUTPUT_FOLDER, 'resultado_liquidaciones.xlsx'))
resultado_final.to_excel(os.path.join(OUTPUT_FOLDER, 'resultado_final.xlsx'))
maestra_resultados.to_excel(os.path.join(OUTPUT_FOLDER, 'maestra_resultados.xlsx'))

In [ ]:
maestra_resultados_por_empleado_centro_costo[
    (maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '5002899')
]

## Generación de PDFs

In [ ]:
def get_dataframes(
        df_resultado_detallado,
        df_resultado_liquidaciones
):
    resultado_liquidaciones = df_resultado_liquidaciones.set_index(['CodigoEmpleado', 'Fecha'])
    resultado_liquidaciones.fillna(value={
        'PagoReal': 0,
        'Liquidacion': 0
    }, inplace=True)

    resultado_detallado = df_resultado_detallado.fillna(value={
        'FactorIncentivo': 0,
        'PorcentajeCumplimiento': 0,
        'Presupuesto': 0,
        'Real': 0,
        'Liquidado': 0,
        'ResultadoPresupuesto': 0,
        'ResultadoRealMes': 0,
        'PorcentajeCumplimientoMes': 0
    })

    return resultado_detallado, resultado_liquidaciones

In [ ]:
df_detalles_para_pdf, df_liquidaciones_para_pdf = get_dataframes(
    df_resultado_detallado=maestra_resultados_por_empleado,
    df_resultado_liquidaciones=resultado_liquidaciones
)